In [1]:
suppressPackageStartupMessages(suppressWarnings(library(ggplot2)))
suppressPackageStartupMessages(suppressWarnings(library(dplyr)))
suppressPackageStartupMessages(suppressWarnings(library(tidyr)))


In [2]:
cell_type = "PBMC"


In [3]:
# path set
input_file_path <- file.path(paste0("../results/","regression/",cell_type))
# read in the data
output_path <- file.path(paste0("../figures/","regression/",cell_type,"/"))
# create output directory if it doesn't exist
dir.create(output_path, recursive = TRUE, showWarnings = FALSE)


In [4]:
## function to process the data for visualization
process_subset_data <- function(data_path){
    # read in the data
    data <- read.csv(data_path, header = TRUE, sep = ",", stringsAsFactors = FALSE)
    # get the basename of the files

    data <- data %>%
        dplyr::arrange(desc(abs(coefficients))) %>%
        tidyr::separate(
            feature_names,
            into = c(
                "compartment",
                "feature_group",
                "measurement",
                "channel", 
                "parameter1", 
                "parameter2"
            ),
            sep = "_",
            remove = FALSE
        ) %>%
        dplyr::mutate(channel_cleaned = channel) %>%
        dplyr::arrange(desc(abs(coefficients)))

    # Clean channel for visualization
    data$channel_learned <- dplyr::recode(data$channel,
            "CorrDNA" = "nuclei",
            "CorrMito" = "Mito",
            "CorrER" = "ER",
            "CorrGasdermin" = "gasdermin",
            "CorrPM" = "PM",
            .default = "other",
            .missing="other"
    )
    data <- data %>%
        dplyr::group_by(feature_group, channel_learned, compartment) %>%
        dplyr::slice_max(order_by = coefficients, n = 1)
    return(data)
}


In [5]:
# get all files in a directory
files <- list.files(path = input_file_path, pattern = "*.csv", full.names = TRUE)
coef_gg_file <- file.path(paste0(output_path,"/","top_abs_val_coefficients_enet.pdf"))
pdf(file=coef_gg_file, width=7, height=4)
for (i in files){
    filename <- basename(i)
    # split the string at the first _
    filename <- strsplit(filename, "_", fixed = TRUE)[[1]]
    cytokine <- filename[1]
    shuffle <- filename[2]
    # preprocess the data
    data <- process_subset_data(i)
    # plot the data
    coef_gg <- (
        ggplot(data, aes(x = channel_learned, y = feature_group))
        + geom_point(aes(fill = abs(coefficients)), pch = 22, size = 6)
        + facet_wrap("~compartment", ncol = 3)
        + theme_bw()
        + scale_fill_continuous(
            name="Top Abs. val\ntreatment\nlinear model\ncoefficient",
            low = "darkblue",
            high = "yellow",
        )
        + xlab("Channel")
        + ylab("Feature")
        + theme(
            axis.text = element_text(size = 10),
            axis.title = element_text(size = 10),
            title = element_text(size = 14),
            legend.title = element_text(size = 12),
            legend.text = element_text(size = 12),
        )
        # rotate x axis labels
        + theme(axis.text.x = element_text(angle = 90, hjust = 1))
        + ggtitle(paste0("Top Abs. val treatment ElasticNet coefficients for \n",cytokine,"\n",shuffle," model"))
        + theme(plot.title = element_text(hjust = 0.5))
    )
    plot(coef_gg)
}
dev.off()


Warning message:
“Expected 6 pieces. Additional pieces discarded in 228 rows [3, 16, 17, 21, 25,
30, 34, 36, 43, 51, 55, 60, 61, 63, 64, 70, 71, 73, 75, 76, ...].”
Warning message:
“Expected 6 pieces. Missing pieces filled with `NA` in 323 rows [1, 2, 4, 5, 6,
7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 22, 23, 24, ...].”
Warning message:
“Expected 6 pieces. Additional pieces discarded in 228 rows [1, 6, 17, 18, 19,
20, 21, 22, 23, 41, 42, 43, 44, 45, 381, 382, 383, 384, 385, 386, ...].”
Warning message:
“Expected 6 pieces. Missing pieces filled with `NA` in 323 rows [3, 4, 7, 8, 9,
10, 11, 12, 13, 14, 15, 16, 24, 25, 26, 27, 28, 29, 30, 31, ...].”
Warning message:
“Expected 6 pieces. Additional pieces discarded in 228 rows [6, 16, 18, 19, 20,
21, 22, 23, 24, 44, 45, 47, 66, 381, 382, 383, 384, 385, 386, 387, ...].”
Warning message:
“Expected 6 pieces. Missing pieces filled with `NA` in 323 rows [1, 2, 3, 4, 5,
8, 9, 10, 11, 12, 13, 14, 17, 25, 26, 27, 28, 29, 30, 31, ...].”
Warning m

png 
  2